### Notebook to train and test Mask R-CNN on your own dataset.

##### All file paths need to be changed to direct to your data.

In [ ]:
### install dependencies:

# might need to install opencv
!pip3 install Cython cupy-cuda112 cupy-cuda102   # Probably only need one of these but it works so I'm gonna leave it for now.
!pip3 install pyyaml==5.1

# Torch 1.8.1 does not work despite being the latest stable release. As such, use 1.7. 
# Cuda version on MAGEOHub is currently 11.3 - cu110 all works fine.

# To be tried once the bug with torch 1.8 has been fixed 
#!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

In [ ]:
# Import torch and check versions and cuda availability, and resources
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
!nvcc --version
!nvidia-smi

from IPython.display import display, clear_output

In [ ]:
# Some basic imports:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# tensorboard?
%load_ext tensorboard
%tensorboard --logdir output

# necessary libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json

#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
### The following two cells just determine that the setup is working correctly on a standard example.

# Get image, hash wget line if already downloaded.
# Just unhash if running this code for the first time, i.e. input.jpg has not yet been downloaded

!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg
im = cv2.imread("./input.jpg")

# Create config
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))

#cfg.merge_file = 'https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
#cfg.merge_from_file("./detectron2/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# Bit worried this line won't work for everyone - if it doesn't, go to https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md to find a link to the model weights

#cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"

# Here we just get the pre-trained weights straight from facebook hosting website

cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl'



# Create predictor
predictor = DefaultPredictor(cfg)

# Make prediction
outputs = predictor(im)

In [ ]:
# Let's have a look
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))

In [ ]:
# Make sure the inevitable error messages are useful.
CUDA_LAUNCH_BLOCKING="1"

In [ ]:
#### This loop creates training data from pngs and GeoJSON files of manually delineated crowns

def get_tree_dicts(directory):
    classes = ['tree']
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.geojson')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)
        
        
        
        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        # Make sure we have the correct height and width
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["height"] = height
        record["width"] = width
        record["image_id"] = filename[0:400]
        print(filename[0:400])
            
        

        objs = []
        for features in img_anns['features']:
            anno = features['geometry']
            # print("##### HERE IS AN ANNO #####", anno)...weirdly sometimes (but not always) have to make 1000 into a np.array
            px = [a[0] for a in anno['coordinates'][0]]
            py = [np.array(height) - a[1] for a in anno['coordinates'][0]]
            # print("### HERE IS PY ###", py)
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            # print("#### HERE ARE SOME POLYS #####", poly)
            obj = {
                   "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                   "bbox_mode": BoxMode.XYXY_ABS,
                   "segmentation": [poly],
                   "category_id": 0,
                   "iscrowd": 0
                   }
            objs.append(obj)
            # print("#### HERE IS OBJS #####", objs)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "test"]:
    DatasetCatalog.register("trees_" + d, lambda d=d: get_tree_dicts('/home/jovyan/lustre_scratch/paracou_data/high_integrity_data/' + d))
    MetadataCatalog.get("trees_" + d).set(thing_classes=['tree'])
trees_metadata = MetadataCatalog.get("trees_train")

In [ ]:
# Let's look at our training images and annotations for our dataset!

dataset_dicts = get_tree_dicts("/home/jovyan/lustre_scratch/paracou_data/high_integrity_data/train")
for d in random.sample(dataset_dicts, 15):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=trees_metadata, scale=0.4)
    out = visualizer.draw_dataset_dict(d)
    image = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
    #display(Image.fromarray(img))
    display(Image.fromarray(image))

In [ ]:
### Setup the evaluator

from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


In [ ]:
# Further setting up the evaluator.

from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader, build_detection_train_loader
import detectron2.utils.comm as comm
#import torch
import time
import datetime

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [ ]:
# Define the config file.

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("trees_train",)
cfg.DATASETS.TEST = ("trees_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
# Define a custom trainer, which will also print evaluations.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("eval_2", exist_ok=True)
            output_folder = "eval_2"
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[T.Resize((800, 800)), T.RandomBrightness(0.8, 1.8), 
        T.RandomContrast(0.6, 1.3), T.RandomSaturation(0.8, 1.4), T.RandomRotation(angle=[90, 90], expand=False), T.RandomLighting(0.7),
        T.RandomFlip(prob=0.4, horizontal=False, vertical=True)]))

In [ ]:
# Train! Can change hyperparameters e.g. learning rate.
# 2000 iterations will take approximately 20 minutes using a single GPU.

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("trees_train",)
cfg.DATASETS.TEST = ("trees_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### From here is the important bit that hasn't been repeated further up
cfg.TEST.EVAL_PERIOD = 10


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


#eval_results = inference_on_dataset(trainer.model, val_loader, DatasetEvaluators(evaluator))

In [ ]:
### Plot training and validation loss on the same plot to see the ideal number of iterations of training.

import json
import matplotlib.pyplot as plt

experiment_folder = '/home/jovyan/detectron2_live/train_test/output'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + '/metrics.json')

plt.plot(
    [x['iteration'] for x in experiment_metrics if 'validation_loss' in x], 
    [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x], label='Total Validation Loss', color='red')
plt.plot(
    [x['iteration'] for x in experiment_metrics if 'total_loss' in x], 
    [x['total_loss'] for x in experiment_metrics if 'total_loss' in x], label='Total Training Loss')

plt.legend(loc='upper right')
plt.title('Comparison of the training and validation loss of Mask R-CNN')
plt.ylabel('Total Loss')
plt.xlabel('Number of Iterations')
plt.show()

In [ ]:
# Setup to predict on new images, here setting up for the trees_test dataset, but can also use this setup
# for predicting on individual images as seen 2 cells down

from detectron2.utils.visualizer import ColorMode

# Weights automatically saved to OUTPUT_DIR + model_final.pth following training
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
cfg.DATASETS.TEST = ("trees_test",)
predictor = DefaultPredictor(cfg)

In [ ]:
# Let's look at our prediction on our test dataset.

dataset_dicts = get_tree_dicts("/home/jovyan/lustre_scratch/paracou_data/high_integrity_data/test")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], metadata=trees_metadata, scale=0.7, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
    display(Image.fromarray(image))

In [ ]:
# make and visualise predictions on a particular image

im = cv2.imread("/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_2014_rgb_tiles_pngs/tile_602400_646900.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], metadata=trees_metadata, scale=1.5, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))

In [ ]:
### Need these packages

!pip install Fiona
!pip install rasterio

In [ ]:
import rasterio
from rasterio.transform import from_origin

In [ ]:
### Reading in dataset dicts for all our areas of interest

dataset_dicts = get_tree_dicts("/home/jovyan/lustre_scratch/paracou_data/high_integrity_data/test")
for d in dataset_dicts:
    print(d["file_name"][-10:-4])

In [ ]:
for filepath in glob.iglob('/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_2014_rgb_tiles_pngs/*.png'):
    #print(filepath)
    print(filepath[-17:-4])

In [ ]:
# This cell outputs Mask R-CNN's predictions as individual rasters

import glob

for filepath in glob.iglob('/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_2014_rgb_tiles_pngs/*.png'):
    print(filepath)
    img = cv2.imread(filepath)
    outputs = predictor(img)
    mask_array = outputs['instances'].pred_masks.cpu().numpy()
    
    
    #print(mask_array.shape)
    # Shifting axes...not sure why...to investigate
    #mask_array = np.moveaxis(mask_array, 0, -1)
    #print(mask_array.shape)
    # here are the shapes for all the different predictions on all the different images
    #print(mask_array.shape)

    num_instances = mask_array.shape[0]
    #print(num_instances)
    mask_array_instance = []
    # black (think this putting zeroes where there are zeroes in our image??? NEED TO EXAMINE)
    output = np.zeros_like(mask_array) 
    #print('output',output.shape)

    mask_array_instance.append(mask_array)
    output = np.where(mask_array_instance[0] == True, 255, output)
    #print(output)
    #print(mask_array_instance)
    fresh_output = output.astype(np.float)
    #print(fresh_output.shape)
    x_scaling = 200/fresh_output.shape[1]
    y_scaling = 200/fresh_output.shape[2]
    # this is an affine transform. This needs to be altered significantly.
    transform = from_origin(int(filepath[-17:-11]), int(filepath[-10:-4])+200, y_scaling, x_scaling)

    new_dataset = rasterio.open('/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_predictions/10_predicted_polygons_'+filepath[-17:-4]+'.tif', 'w', driver='GTiff',
                                height = fresh_output.shape[1], width = fresh_output.shape[2], count = fresh_output.shape[0],
                                dtype=str(fresh_output.dtype),
                                crs='+proj=utm +zone=50 +datum=WGS84 +units=m +no_defs',  
                                transform=transform)

    new_dataset.write(fresh_output)
    new_dataset.close()

### Working fantastically, 30/6/21.


In [ ]:
### This cell outputs large rasters WITH THE NUMBER OF LAYERS = NUMBER OF PREDS
### stacks the raster predictions
### this creates rasters of shape (number of preds, height, width)

dataset_dicts = get_tree_dicts("/home/jovyan/lustre_scratch/paracou_data/high_integrity_data/test")
for d in dataset_dicts:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    mask_array = outputs['instances'].pred_masks.cpu().numpy()
    
    
    #print(mask_array.shape)
    # Shifting axes...not sure why...to investigate
    #mask_array = np.moveaxis(mask_array, 0, -1)
    #print(mask_array.shape)
    # here are the shapes for all the different predictions on all the different images
    #print(mask_array.shape)

    num_instances = mask_array.shape[0]
    #print(num_instances)
    mask_array_instance = []
    # black (think this putting zeroes where there are zeroes in our image??? NEED TO EXAMINE)
    output = np.zeros_like(mask_array) 
    #print('output',output.shape)

    mask_array_instance.append(mask_array)
    output = np.where(mask_array_instance[0] == True, 255, output)
    #print(output)
    #print(mask_array_instance)
    fresh_output = output.astype(np.float)
    #print(fresh_output.shape)
    x_scaling = 100/fresh_output.shape[1]
    y_scaling = 100/fresh_output.shape[2]
    # this is an affine transform. This needs to be altered significantly.
    transform = from_origin(int(d["file_name"][-17:-11]), int(d["file_name"][-10:-4])+100, y_scaling, x_scaling)

    new_dataset = rasterio.open('10_predicted_polygons_'+d["file_name"][-32:-4]+'.tif', 'w', driver='GTiff',
                                height = fresh_output.shape[1], width = fresh_output.shape[2], count = fresh_output.shape[0],
                                dtype=str(fresh_output.dtype),
                                crs='+proj=utm +zone=22 +datum=WGS84 +units=m +no_defs',  
                                transform=transform)

    new_dataset.write(fresh_output)
    new_dataset.close()




In [ ]:
### It reads in a stack raster of all the predictions, and outputs n shapefiles, where n 
### is the number of predictions for that particular image. 
### We can then merge all these predictions. 

import numpy as np
import fiona
import rasterio
import rasterio.features
from shapely.geometry import shape, mapping
from shapely.geometry.multipolygon import MultiPolygon
#from shapely.geometry import Polygon


# Read input band with Rasterio
for filepath in glob.iglob('/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_predictions/*.tif'):
    print(filepath)
    with rasterio.open(filepath) as src:
        shp_schema = {'geometry': 'MultiPolygon','properties': {'pixelvalue': 'int'}}
    
        crs = src.crs
        for i in range(src.count):
            src_band = src.read(i+1)
            src_band = np.float32(src_band)
            #print(src_band.dtype)
            # Keep track of unique pixel values in the input band
            unique_values = np.unique(src_band)
            # Polygonize with Rasterio. `shapes()` returns an iterable
            # of (geom, value) as tuples
            shapes = list(rasterio.features.shapes(src_band, transform=src.transform))




# Get a list of all polygons for a given pixel value
# and create a MultiPolygon geometry with shapely.
# Then write the record to an output shapefile with fiona.
# We make use of the `shape()` and `mapping()` functions from
# shapely to translate between the GeoJSON-like dict format
# and the shapely geometry type.
##### Don't forget to add this folder here!
            with fiona.open('/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes/10_predicted_polygons_'+filepath[-17:-4]+'_'+str(i)+'.shp', 'w', 'ESRI Shapefile', shp_schema, crs) as shp:
                for pixel_value in unique_values:
                    polygons = [shape(geom) for geom, value in shapes
                                if value == pixel_value]
                    multipolygon = MultiPolygon(polygons)
                    shp.write({
                        'geometry': mapping(multipolygon),
                        'properties': {'pixelvalue': int(pixel_value)}
                            })

In [ ]:
import zipfile as zf
import shutil

In [ ]:
### zip up the folder to download

shutil.make_archive("/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes", "zip", "/home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes")

#zip -r /home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes.zip /home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes

In [ ]:
### EOF